# 1) Preparación previa

### Carga de librerías

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

### Lectura del dataset

Se decidió utilizar unos datos de votos legislativos en EEUU, que describe si se votó afirmativamente, negativamente o absteniéndose en 16 leyes.

In [ ]:
df_votes = pd.read_csv('https://raw.githubusercontent.com/Agustin-Bulzomi/Projects/main/Programming/Digital%20House%20(Python)/Support%20Files/Project%203/House_votes_1984.csv')

### Vista general

Se corren varias funciones para obtener un resumen general del dataset

In [ ]:
df_votes.info()

In [ ]:
df_votes.head(10)

# 2) Análisis exploratorio

Se comienza con el análisis general: ¿cómo se distribuye la variable target? ¿están balanceados los datos?

In [ ]:
balance = pd.concat([df_votes['Class Name'].value_counts(), df_votes['Class Name'].value_counts(normalize=True).round(2)], axis = 1)
balance.columns = ['Votos', 'Proporción']
balance

In [ ]:
# Se crea una paleta con los típicos colores partidarios, ya que será usada a futuro
color_partidos = ['red', 'blue']

g = sns.catplot(x='Class Name', 
                    data=df_votes, kind="count", palette=color_partidos,
                    height=3, aspect=.8);

Hay un importante desbalance considerando que casi 2 tercios de los legisladores son demócratas y 1 tercio republicano. Este dato es importante a la hora de contrastar votos. A continuación se procederá con el análisis más específico, ley por ley. Para ello, se crea una nueva columna con el nombre Target ya que es necesaria para realizar análisis con una variable numérica

In [ ]:
df_votes['Target'] = np.where(df_votes['Class Name'] == 'democrat', 1, 0)

In [ ]:
laws = [' handicapped-infants', ' water-project-cost-sharing',
       ' adoption-of-the-budget-resolution', ' physician-fee-freeze',
       ' el-salvador-aid', ' religious-groups-in-schools',
       ' anti-satellite-test-ban', ' aid-to-nicaraguan-contras', ' mx-missile',
       ' immigration', ' synfuels-corporation-cutback', ' education-spending',
       ' superfund-right-to-sue', ' crime', ' duty-free-exports',
       ' export-administration-act-south-africa']

for law in laws:
    numero = laws.index(law) + 1
    display(Markdown('## {} {} {} {} \n #### {}'.format("Ley N°", numero, ": ", law, "Cantidad de votos afirmativos, negativos y abstenciones:")))
    porcentajes = df_votes[law].value_counts()
    print(porcentajes)
    print() # Un espacio para darle legibilidad
    display(Markdown('#### {} '.format("Porcentaje de votos según partido:")))
    table = pd.concat([100*(df_votes.groupby(law)['Target'].mean().round(2)), 100* (1 - (df_votes.groupby(law)['Target'].mean().round(2)))], axis = 1)
    table.columns = ["Democrat", "Republican"]
    display(table)
    display(Markdown('#### {} '.format("Gráfico de la proporción:")))
    grafico = sns.catplot(x = law, hue = "Class Name", data = df_votes, kind="count", palette = color_partidos, height = 3, aspect = .8);
    plt.show(grafico)
    print() # Un espacio para darle legibilidad

# 3) Exportación

Se prepara la base de datos antes de ser exportada para aplicar modelos en otra notebook. Ya la variable target tiene su columna numérica homónima. Ahora se procede a reemplazar los valores string de los votos por otros numéricos (n = -1, ? = 0,  y =  1)

In [ ]:
votos_replace = df_votes.replace(to_replace = ["n", "?", "y"], value = [-1, 0, 1])
votos_replace.head(10)

Se exporta la base de datos

In [ ]:
votos_replace.to_csv('votos_final.csv', index = False)